This notebook is associated with the paper "The relative class number one problem for function fields, I" by K.S. Kedlaya. It runs in SageMath (tested using version 9.5); it also requires the pandas and openpyxl libraries (`sage --pip install openpyxl/pandas`).

In this notebook, we run the exhaustion over Weil polynomials for $q=2$; this requires the input file "curves.txt" produced by "Curves by genus.ipynb". Allow 40 minutes for completion.

In [1]:
load("../Shared/weil_poly_utils.sage")
load("../Shared/function_fields.sage")

# Setup

Bounds on rational points on curves over $\mathbb{F}_2$ and $\mathbb{F}_4$.

In [2]:
point_count_bounds = [(3,5), (5,9), (6,10), (7,14), (8,15), (9, 17), \
                      (10, 20), (10, 21), (11, 23), (12, 26), (13, 27)]

Table of Weil polynomials of simple abelian varieties of order 1 and dimension at most 6, taken from LMFDB.

In [3]:
P.<T> = QQ[]
simple_poly = {'1.2.ac': T^2 - 2*T + 2,
               '2.2.a_ae': T^4 - 4*T^2 + 4,
               '2.2.ad_f': T^4 - 3*T^3 + 5*T^2 - 6*T + 4,
               '2.2.ac_c': T^4 - 2*T^3 + 2*T^2 - 4*T + 4,
               '2.2.ab_ab': T^4 - T^3 - T^2 - 2*T + 4,
               '3.2.ad_c_b': T^6 - 3*T^5 + 2*T^4 + T^3 + 4*T^2 - 12*T + 8,
               '3.2.ae_j_ap': T^6 - 4*T^5 + 9*T^4 - 15*T^3 + 18*T^2 - 16*T + 8,
               '4.2.af_m_au_bd': T^8 - 5*T^7 + 12*T^6 - 20*T^5 + 29*T^4 - 40*T^3 + 48*T^2 - 40*T + 16,
               '4.2.ae_g_ae_c': T^8 - 4*T^7 + 6*T^6 - 4*T^5 + 2*T^4 - 8*T^3 + 24*T^2 - 32*T + 16,
               '4.2.ad_c_a_b': T^8 - 3*T^7 + 2*T^6 + T^4 + 8*T^2 - 24*T + 16,
               '4.2.ae_f_c_al': T^8 - 4*T^7 + 5*T^6 + 2*T^5 - 11*T^4 + 4*T^3 + 20*T^2 - 32*T + 16,
               '4.2.ae_e_h_av': T^8 - 4*T^7 + 4*T^6 + 7*T^5 - 21*T^4 + 14*T^3 + 16*T^2 - 32*T + 16,
               '4.2.af_n_az_bn': T^8 - 5*T^7 + 13*T^6 - 25*T^5 + 39*T^4 - 50*T^3 + 52*T^2 - 40*T + 16,
               '6.2.ag_p_av_y_abn_cn': T^12 - 6*T^11 + 15*T^10 - 21*T^9 + 24*T^8 - 39*T^7 + 65*T^6 - 78*T^5 + 96*T^4 - 168*T^3 + 240*T^2 - 192*T + 64,
               '6.2.af_j_ah_d_ab_ab': T^12 - 5*T^11 + 9*T^10 - 7*T^9 + 3*T^8 - T^7 - T^6 - 2*T^5 + 12*T^4 - 56*T^3 + 144*T^2 - 160*T + 64,
               '6.2.ag_p_at_g_bb_acj': T^12 - 6*T^11 + 15*T^10 - 19*T^9 + 6*T^8 + 27*T^7 - 61*T^6 + 54*T^5 + 24*T^4 - 152*T^3 + 240*T^2 - 192*T + 64
              }

Generate Weil polynomials corresponding to simple abelian varieties of order 1 of larger dimension.

In [4]:
def generate_poly(n):
    P.<alpha,eta> = QQ[]
    u = alpha^2 + (eta-1)*alpha - 2*eta
    v = cyclotomic_polynomial(n)(eta)
    w = u.resultant(v, eta)
    Q.<T> = QQ[]
    return w(T, 0)

In [5]:
simple_pols = {g: [u for u in simple_poly.values() if u.degree() == 2*g] for g in range(1, 7)}

In [6]:
for n in range(1, 500):
    t = euler_phi(n)
    if t >= 8 and t <= 18 and n != 30:
        if t not in simple_pols:
            simple_pols[t] = []
        simple_pols[t].append(generate_poly(n)(T))

Generate Weil polynomials corresponding to arbitrary abelian varieties of order 1.

In [7]:
pols = {}
l = list(simple_pols.keys())
temp = []
degs = []
for i in l:
    temp += simple_pols[i]
    degs += [i] * len(simple_pols[i])
for g in range(1,17):
    vects = WeightedIntegerVectors(g, degs)
    pols[g] = [prod(temp[i]**v[i] for i in range(len(v))) for v in vects]
assert all(u.degree() == 2*g for g in simple_pols for u in simple_pols[g])

In [8]:
P.<T> = QQ[]
Q.<U,V> = QQ[]
pols_extended = {}
for m in range(2,3):
    for i in range(1,7):
        pols_extended[i,m] = [(u(U).resultant(U^m-V))(0,T) for u in pols[i]]

Generate Weil polynomials with specified dimension and point counts over $\mathbb{F}_2$ and $\mathbb{F}_4$,
then filter for positivity and the resultant criterion.

In [9]:
weil_polys_by_counts = {}

def generate_weil_polys_by_counts(g, i, j):
    if (i-j)%2:
        return []
    if (g, i, j) not in weil_polys_by_counts:
        P.<T> = QQ[]
        Q.<t> = PowerSeriesRing(QQ)
        v = i*t + j*t^2/2
        u = (exp(v)*(1-t)*(1-2*t)).list()
        if g == 1:
            if u[2] != 2:
                l = []
            else:
                l = P.weil_polynomials(q=2,d=2*g,lead=[ZZ(_) for _ in u[:2]])
        else:
            l = P.weil_polynomials(q=2,d=2*g,lead=[ZZ(_) for _ in u[:3]])
        weil_polys_by_counts[g, i, j] = []
        for w in l:
            if check_curve_positivity(w, 8) and not _nojac_serre(w):
                weil_polys_by_counts[g, i, j].append(w)
    return weil_polys_by_counts[g, i, j]

Exhaust over Weil polynomials based on the bounds on $\#C(\mathbb{F}_2)$ and $\#C(\mathbb{F}_4)$. If $d$ is specified, we add extra constraints based on the value of $d$. Additional optional arguments:
- `hyperelliptic`: assume that $C'$ is hyperelliptic (only relevant if $d=2$).
- `verbose`: print additional internal data.
- `blocklist`: if not None, must be a dictionary in which `blocklist[d]` is a list of Weil polynomials known not to occur for Jacobians of curves of genus `d`. (The similar term "blacklist" is deprecated for social justice reasons.)
- `real`: if True, returns real Weil polynomials instead of ordinary Weil polynomials (for shorter output).
- `update`: if True, updates `candidate_polys` as a side effect.
- `easy`: if True, uses only the "easy" criteria.

In [10]:
candidate_polys = {}

In [11]:
def count_bounds(g,g1,d=None,hyperelliptic=False,verbose=False,
                 blocklist=None,real=False,update=False,easy=False):
    # Identify L-polynomials for A with suitably small point counts and excess.
    delta = g1 - d*g + d-1 if d else None
    bound1 = 6 if (d==2 and hyperelliptic) else point_count_bounds[g][0]
    bound2 = 10 if (d==2 and hyperelliptic) else point_count_bounds[g][1]
    pols2 = []
    for u1 in pols[g1-g]:
        tmp = trace_from_weil_poly(u1, 4)
        c = tmp[0]+tmp[1]-delta if d == 2 else tmp[1]-tmp[0]
        if tmp[0] <= bound1 and c+tmp[0] <= bound2 and \
            (tmp[0] + 0.3366*c + 0.1137*(tmp[2]-tmp[0]) + 0.0537*(tmp[3]-tmp[1]) <= 0.8042*g + 5.619):
            pols2.append(u1)
    if verbose:
        print('Number of polynomials for A: {}'.format(len(pols2)))
    if not pols2:
        return {}
    # Identify possible first two point counts for C.
    l = {}
    for u1 in pols2:
        tmp = trace_from_weil_poly(u1, 4)
        if d == 2 and not hyperelliptic:
            tmp2 = min(bound1, tmp[0] + delta)
        else:
            tmp2 = bound1
        for i in range(tmp[0], tmp2+1):
            for j in range(2*tmp[0]+tmp[1]-delta if d == 2 else tmp[0], bound2+1):
                if (i-j) % 2 == 0:
                    l[i,j] = []
    if verbose:
        print(l)
    # Compile L-polynomials for C, sorted by their first two point counts.
    for (i,j) in l:
        l[i,j] = generate_weil_polys_by_counts(g, i, j)
    # If d=2 and delta=0, filter for required divisibility in #J(C)(F_2).
    if d == 2 and delta==0:
        l = {(i,j): [u for u in l[i,j] if u(1)%2 == 0] for (i,j) in l}
    l = {(i,j): l[i,j] for (i,j) in l if l[i,j]}    
    if verbose:
        print('Number of polynomials for C: {}'.format(sum(len(l[i,j]) for (i,j) in l)))
    if not l:
        return {}
    # For each possible Weil polynomial for C, run over choices for A.
    traces = {u1: trace_from_weil_poly(u1, 12) for u1 in pols2}
    l2 = {}
    for (i,j) in l:
        for u in l[i,j]:
            # Check C against the blocklist.
            if blocklist and ((g in blocklist and u in blocklist[g]) or _nojac_serre(u, blocklist=blocklist)):
                continue
            u_real = prod(f for (f, _) in u.trace_polynomial()[0].factor())
            tmp = point_count_from_weil_poly(u, 12)
            tmp3 = []
            for u1 in pols2:
                u1_real = prod(f for (f, _) in u1.trace_polynomial()[0].factor())
                tmp2 = [tmp[i] - traces[u1][i] for i in range(12)]
                # Early abort (based on positivity) using i and j.
                if not (i >= traces[u1][0] and j >= traces[u1][1]):
                    continue
                # Early abort for d=2.
                if d == 2 and not (j >= 2*traces[u1][0] + traces[u1][1] - delta):
                    continue
                # Check positivity and the resultant condition for C'.
                v  = u*u1
                if (not check_curve_positivity(v, 12)) or _nojac_serre(v, blocklist=blocklist):
                    continue
                # Check C' against the blocklist.
                if blocklist and g1 in blocklist and v in blocklist[g1]:
                    continue
                # Check: if delta=1, C'(F_2) >= 1; if delta=2, C'(F_4) >= 1.        
                if delta == 1:
                    if not (tmp2[0] >= 1):
                        continue
                elif not easy and delta == 2:
                    if not (tmp2[1] >= 1):
                        continue
                # Check resultant condition for a degree-d cover.
                if not easy and d and gcd(d, modified_reduced_resultant(u_real, u1_real)) == 1:
                    continue
                # Check conditions depending on d.
                if d == 2:
                    # Parity constraint: if delta=0, #J(C)(F_2) is even.
                    if delta == 0 and u(1)%2:
                        continue
                    # Deuring-Shafarevich constraints.
                    ds_ram = v.newton_slopes(2).count(0) - 2*u.newton_slopes(2).count(0)+1
                    if not (ds_ram in ([delta] if delta in [0,1] else range(1,delta+1))):
                        continue
                    if not all(tmp2[2*i-1] >= 2*tmp[i-1]-ds_ram for i in range(1,4)):
                        continue
                    if not easy and ds_ram == 1 and not (tmp2[0] >= 1):
                        continue
                    if not easy and ds_ram == 2 and not (tmp2[1] >= 2):
                        continue
                    # Upper bound on #C'(F_2) based on delta when C' is not hyperelliptic.
                    if not easy and not (hyperelliptic or (tmp2[0] in range(delta+1))):
                        continue
                    # Positivity and the resultant criterion for the relative quadratic twist.
                    v2 = u*u1(-u1.parent().gen())
                    if not easy and ((not check_curve_positivity(v2, 12)) or _nojac_serre(v2, blocklist=blocklist)):
                        continue
                    # Check the relative quadratic twist against the blocklist.
                    if not easy and blocklist and g1 in blocklist and v2 in blocklist[g1]:
                        continue
                    # If t <= 2, parity condition on odd-degree point counts.
                    if not easy and ds_ram <= 2 and not all((tmp2[i] - tmp2[0])%2 == 0 for i in range(2, 12, 2)):
                        continue
                    if not easy and ds_ram <= 2 and tmp2[2] - tmp2[0] > 2*(tmp[2] - tmp[0]):
                        continue
                if d == 3:
                    if not (tmp[2] >= tmp[0] + 2*traces[u1][0] + traces[u1][2] and \
                            tmp[5] >= tmp[1] + 2*traces[u1][1] + traces[u1][5]):
                        continue
                if d == 4:
                    if not (tmp[3] >= 4*traces[u1][0]  + traces[u1][3] - 2*delta):
                        continue
                # Apply Remark 9.10.
                if (d == 4 and (g,g1) in [(2,6), (3,9)]) or (d == 6 and (g,g1) == (2,7)) or \
                   (d == 4 and (g,g1) == (2,5) and \
                    (u.trace_polynomial()[0], v.trace_polynomial()[0]) not in candidate_polys[2,2,5]):
                    if not easy and gcd(d, modified_reduced_resultant(u_real, u1_real)) == 2:
                        print("Alert: Remark 9.10 applied")
                        continue                    
                # Record this candidate.
                if verbose:
                    print(tmp, point_count_from_weil_poly(v, 12), v(1))
                tmp3.append(u1)
            if tmp3:
                if real:
                    l2[label_from_weil_poly(u)] = (u.trace_polynomial()[0], [v.trace_polynomial()[0] for v in tmp3])
                else:
                    l2[label_from_weil_poly(u)] = (u, tmp3)
    if update:
        if (d, g, g1) not in candidate_polys:
            candidate_polys[d, g, g1] = []
        for s in l2:
            (u, tmp) = l2[s]
            for v in tmp:
                if (u,v) not in candidate_polys[d, g, g1]:
                    candidate_polys[d, g, g1].append((u,v))
    return(l2)

# Excluded Weil polynomials

Define a "blocklist" of Weil polynomials which are known not to occur for Jacobians (based on LMFDB).

In [12]:
not_jac = {
    1: [],
    2: [T^4 - 4*T^3 + 8*T^2 - 8*T + 4],
    3: [T^6 - T^5 + T^4 - 2*T^3 + 2*T^2 - 4*T + 8,
        T^6 - 2*T^5 + 2*T^4 + 4*T^2 - 8*T + 8],
    4: [T^8 - 3*T^7 + 2*T^6 + 6*T^5 - 16*T^4 + 12*T^3 + 8*T^2 - 24*T + 16,
        T^8 - 3*T^7 + 4*T^6 + 2*T^5 - 8*T^4 + 4*T^3 + 16*T^2 - 24*T + 16,
        T^8 - 2*T^7 + 4*T^5 - 8*T^4 + 8*T^3 - 16*T + 16,
        T^8 - 2*T^7 + 3*T^6 - 5*T^5 + 6*T^4 - 10*T^3 + 12*T^2 - 16*T + 16,
        T^8 + 3*T^7 + 5*T^6 + 9*T^5 + 16*T^4 + 18*T^3 + 20*T^2 + 24*T + 16,
        T^8 + 3*T^7 + 6*T^6 + 12*T^5 + 20*T^4 + 24*T^3 + 24*T^2 + 24*T + 16,
        T^8 - 3*T^7 + 6*T^6 - 12*T^5 + 20*T^4 - 24*T^3 + 24*T^2 - 24*T + 16,
        T^8 + T^7 + 2*T^6 + 4*T^5 + 8*T^4 + 8*T^3 + 8*T^2 + 8*T + 16,
        T^8 + T^7 + 2*T^6 + 4*T^5 + 6*T^4 + 8*T^3 + 8*T^2 + 8*T + 16,
        T^8 + 4*T^6 + 11*T^4 + 16*T^2 + 16,
        T^8 + 4*T^6 + 9*T^4 + 16*T^2 + 16,
        T^8 + 4*T^6 - T^5 + 10*T^4 - 2*T^3 + 16*T^2 + 16],
    5: [],
    6: [],
    7: [],
    8: [],
    9: []
}

Add genus 5 data from Dragutinović to the blocklist.

In [13]:
not_jac[5] = [u for u in P.weil_polynomials(10, 2) if check_curve_positivity(u, 5) and \
             not _nojac_serre(u) and label_from_weil_poly(u) not in curves_by_poly[5]]
len(not_jac[5])

4088

# Exhaustion over Weil polynomials (section 6)

Run the exhaustion over Weil polynomials in some cases that yield no results.

In [14]:
assert not count_bounds(6, 17, easy=True)

In [15]:
assert not count_bounds(5, 15, easy=True)

In [16]:
assert not count_bounds(4, 13, easy=True)

In [17]:
assert not count_bounds(3, 11, easy=True)

In [18]:
assert not count_bounds(2, 9, easy=True)

In [19]:
assert not count_bounds(1, 7, easy=True)

Run the exhaustive search in all remaining cases with $g=1$.

In [20]:
count_bounds(1, 6, easy=True)

{'1.2.c': (T^2 + 2*T + 2,
  [T^10 - 5*T^9 + 10*T^8 - 10*T^7 + 5*T^6 - 2*T^5 + 10*T^4 - 40*T^3 + 80*T^2 - 80*T + 32]),
 '1.2.b': (T^2 + T + 2,
  [T^10 - 4*T^9 + 4*T^8 + 4*T^7 - 12*T^6 + 16*T^5 - 24*T^4 + 16*T^3 + 32*T^2 - 64*T + 32])}

In [21]:
count_bounds(1, 5, blocklist=not_jac, easy=True)

{'1.2.b': (T^2 + T + 2,
  [T^8 - 4*T^7 + 4*T^6 + 7*T^5 - 21*T^4 + 14*T^3 + 16*T^2 - 32*T + 16,
   T^8 - 4*T^7 + 6*T^6 - 4*T^5 + 2*T^4 - 8*T^3 + 24*T^2 - 32*T + 16,
   T^8 - 4*T^7 + 4*T^6 + 8*T^5 - 24*T^4 + 16*T^3 + 16*T^2 - 32*T + 16])}

In [22]:
count_bounds(1, 4, blocklist=not_jac, easy=True)

{'1.2.c': (T^2 + 2*T + 2,
  [T^6 - 4*T^5 + 8*T^4 - 12*T^3 + 16*T^2 - 16*T + 8,
   T^6 - 5*T^5 + 13*T^4 - 22*T^3 + 26*T^2 - 20*T + 8]),
 '1.2.a': (T^2 + 2,
  [T^6 - 3*T^5 + 2*T^4 + T^3 + 4*T^2 - 12*T + 8,
   T^6 - 3*T^5 + 3*T^4 - 2*T^3 + 6*T^2 - 12*T + 8])}

In [23]:
count_bounds(1, 3, blocklist=not_jac, easy=True)

{'1.2.ac': (T^2 - 2*T + 2, [T^4 - T^3 - T^2 - 2*T + 4]),
 '1.2.ab': (T^2 - T + 2,
  [T^4 - T^3 - T^2 - 2*T + 4, T^4 - 2*T^3 + 2*T^2 - 4*T + 4]),
 '1.2.b': (T^2 + T + 2,
  [T^4 - 3*T^3 + 5*T^2 - 6*T + 4, T^4 - 4*T^3 + 8*T^2 - 8*T + 4])}

In [24]:
count_bounds(1, 2, easy=True)

{'1.2.a': (T^2 + 2, [T^2 - 2*T + 2]),
 '1.2.b': (T^2 + T + 2, [T^2 - 2*T + 2]),
 '1.2.c': (T^2 + 2*T + 2, [T^2 - 2*T + 2])}

Run the exhaustive search in a few cases with $d=2$ that yield no results.

In [25]:
assert not count_bounds(9, 17, d=2) #long

In [26]:
assert not count_bounds(9, 17, d=2, hyperelliptic=True)

In [27]:
assert not count_bounds(8, 15, d=2) #long

In [28]:
assert not count_bounds(8, 15, d=2, hyperelliptic=True)

In [29]:
assert not count_bounds(7, 14, d=2)

In [30]:
assert not count_bounds(7, 14, d=2, hyperelliptic=True)

In [31]:
assert not count_bounds(6, 13, d=2)

In [32]:
assert not count_bounds(6, 13, d=2, hyperelliptic=True)

In [33]:
assert not count_bounds(6, 12, d=2)

In [34]:
assert not count_bounds(6, 12, d=2, hyperelliptic=True)

In [35]:
assert not count_bounds(5, 11, d=2)

In [36]:
assert not count_bounds(5, 11, d=2, hyperelliptic=True)

In [37]:
assert not count_bounds(5, 10, d=2)

In [38]:
assert not count_bounds(5, 10, d=2, hyperelliptic=True)

In [39]:
assert not count_bounds(4, 11, d=2)

In [40]:
assert not count_bounds(4, 11, d=2, hyperelliptic=True)

In [41]:
assert not count_bounds(4, 10, d=2)

In [42]:
assert not count_bounds(4, 10, d=2, hyperelliptic=True)

In [43]:
assert not count_bounds(4, 9, d=2)

In [44]:
assert not count_bounds(4, 9, d=2, hyperelliptic=True)

In [45]:
assert not count_bounds(3, 9, d=2)

In [46]:
assert not count_bounds(3, 9, d=2, hyperelliptic=True)

In [47]:
assert not count_bounds(3, 8, d=2)

In [48]:
assert not count_bounds(3, 8, d=2, hyperelliptic=True)

In [49]:
assert not count_bounds(3, 7, d=2)

In [50]:
assert not count_bounds(3, 7, d=2, hyperelliptic=True)

In [51]:
assert not count_bounds(2, 6, d=2, hyperelliptic=True)

In [52]:
assert not count_bounds(2, 7, d=2, hyperelliptic=True)

Run the exhaustive search in a few cases with $d=3$ that yield no results.

In [53]:
assert not count_bounds(6, 16, d=3)

In [54]:
assert not count_bounds(5, 14, d=3)

In [55]:
assert not count_bounds(5, 13, d=3)

In [56]:
assert not count_bounds(4, 12, d=3)

In [57]:
assert not count_bounds(4, 11, d=3)

In [58]:
assert not count_bounds(3, 10, d=3)

In [59]:
assert not count_bounds(3, 9, d=3)

In [60]:
assert not count_bounds(3, 8, d=3)

In [61]:
assert not count_bounds(2, 8, d=3)

In [62]:
assert not count_bounds(2, 7, d=3)

Run the exhaustive search in a few cases with $d=4$ that yield no results.

In [63]:
assert not count_bounds(3, 10, d=4)

In [64]:
assert not count_bounds(2, 8, d=4)

In [65]:
assert not count_bounds(2, 7, d=4)

Run the exhaustive search in a few cases with $d \geq 5$ that yield no results.

In [66]:
assert not count_bounds(2, 8, d=5)

In [67]:
assert not count_bounds(2, 7, d=5)

In [68]:
assert not count_bounds(2, 8, d=6)

# More exhaustion (section 9)

Run exhaustive searches in the remaining cases, compiling the results for further analysis.

In [69]:
count_bounds(7, 13, d=2, blocklist=not_jac, real=True, update=True); #long

In [70]:
count_bounds(7, 13, d=2, blocklist=not_jac, hyperelliptic=True, real=True, update=True);

In [71]:
count_bounds(6, 11, d=2, blocklist=not_jac, real=True, update=True);

In [72]:
count_bounds(6, 11, d=2, blocklist=not_jac, hyperelliptic=True, real=True, update=True);

In [73]:
count_bounds(5, 9, d=2, blocklist=not_jac, real=True, update=True);

In [74]:
count_bounds(5, 9, d=2, blocklist=not_jac, hyperelliptic=True, real=True, update=True);

In [75]:
count_bounds(4, 8, d=2, blocklist=not_jac, real=True, update=True);

In [76]:
count_bounds(4, 8, d=2, blocklist=not_jac, hyperelliptic=True, real=True, update=True);

In [77]:
count_bounds(4, 7, d=2, blocklist=not_jac, real=True, update=True);

In [78]:
count_bounds(4, 7, d=2, blocklist=not_jac, hyperelliptic=True, real=True, update=True);

In [79]:
count_bounds(3, 6, d=2, blocklist=not_jac, real=True, update=True);

In [80]:
count_bounds(3, 6, d=2, blocklist=not_jac, hyperelliptic=True, real=True, update=True);

In [81]:
count_bounds(3, 5, d=2, blocklist=not_jac, real=True, update=True);

In [82]:
count_bounds(3, 5, d=2, blocklist=not_jac, hyperelliptic=True, real=True, update=True);

In [83]:
count_bounds(2, 5, d=2, blocklist=not_jac, hyperelliptic=True, real=True, update=True);

In [84]:
count_bounds(2, 4, d=2, blocklist=not_jac, hyperelliptic=True, real=True, update=True);

In [85]:
count_bounds(2, 3, d=2, blocklist=not_jac, hyperelliptic=True, real=True, update=True);

In [86]:
count_bounds(4, 10, d=3, blocklist=not_jac, real=True, update=True);

In [87]:
count_bounds(3, 7, d=3, blocklist=not_jac, real=True, update=True);

In [88]:
count_bounds(2, 6, d=3, blocklist=not_jac, real=True, update=True);

In [89]:
count_bounds(2, 5, d=3, blocklist=not_jac, real=True, update=True);

In [90]:
count_bounds(2, 4, d=3, blocklist=not_jac, real=True, update=True);

In [91]:
count_bounds(3, 9, d=4, blocklist=not_jac, real=True, update=True);

Alert: Remark 9.10 applied
Alert: Remark 9.10 applied
Alert: Remark 9.10 applied


In [92]:
count_bounds(2, 5, d=4, blocklist=not_jac, real=True, update=True);

Alert: Remark 9.10 applied
Alert: Remark 9.10 applied
Alert: Remark 9.10 applied
Alert: Remark 9.10 applied


In [93]:
count_bounds(2, 6, d=4, blocklist=not_jac, real=True, update=True);

Alert: Remark 9.10 applied
Alert: Remark 9.10 applied


In [94]:
count_bounds(2, 6, d=5, blocklist=not_jac, real=True, update=True);

In [95]:
count_bounds(2, 7, d=6, blocklist=not_jac, real=True, update=True);

Alert: Remark 9.10 applied
Alert: Remark 9.10 applied
Alert: Remark 9.10 applied
Alert: Remark 9.10 applied


In [96]:
count_bounds(2, 8, d=7, blocklist=not_jac, real=True, update=True);

In [97]:
sum(len(l) for l in candidate_polys.values())

207

In [98]:
len(candidate_polys[2,7,13])+len(candidate_polys[2,6,11])

40

# Populate the spreadsheet

Write output to an Excel spreadsheet.

In [99]:
l = []
for (d, g, g1) in candidate_polys:
    for (u, v) in candidate_polys[d, g, g1]:
        u1 = u.reciprocal_transform(q=2)
        v1 = v.reciprocal_transform(q=2)
        ct1 = point_count_from_weil_poly(u1, 13)
        ct2 = point_count_from_weil_poly(u1*v1, 13)
        t = [d, g, g1, ct1, ct2, \
             label_from_weil_poly(u.reciprocal_transform(q=2)), u1(1), u1(1)*u1(-1)]
        l.append(t)
l.sort()

In [100]:
import pandas
di = {"d": [],
     "g": [],
     "g'": [],
     "Label of J(C)": [],
     "#J(C)(F_2)": [],
     "#J(C)(F_4)": [],
     "Counts of C": [],
     "Counts of C'": []
     }
for t in l:
    di["d"].append(t[0])
    di["g"].append(t[1])
    di["g'"].append(t[2])
    di["Counts of C"].append(t[3])
    di["Counts of C'"].append(t[4])
    di["Label of J(C)"].append(t[5])
    di["#J(C)(F_2)"].append(t[6])
    di["#J(C)(F_4)"].append(t[7])
for s in di.keys():
    di[s] = pandas.Series(di[s], dtype='object')

In [101]:
df = pandas.DataFrame(di)
df.to_excel('../Shared/polys.xlsx')